In [1]:
import torch
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

trainDataset = torchvision.datasets.FashionMNIST("fashionDataset", train=True, download=True, transform=transforms.ToTensor())
testDataset = torchvision.datasets.FashionMNIST("fashionTrain", train=False, download=True, transform=transforms.ToTensor())


x,y = trainDataset[0]
print(y)

cuda
9


In [2]:
class Encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, 3, 1, 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2)
        )
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, 5, 1, 2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2)
        )
        self.conv3 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 50, 5, 1, 2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2)
        )
        self.fc1 = torch.nn.Sequential(
            torch.nn.Linear(50*3*3, 100),
            torch.nn.ReLU(),
            torch.nn.Linear(100, 10)
        )
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.resize(-1, 50*3*3)
        x = self.fc1(x)
        return x
    
class Decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Sequential(
            torch.nn.Linear(10, 100),
            torch.nn.ReLU(),
            torch.nn.Linear(100, 50*3*3)
        )
        self.conv1 = torch.nn.Sequential(
            torch.nn.Upsample(scale_factor=2, mode='bicubic'),
            torch.nn.ConvTranspose2d(50, 32, 5, 1, 2),
            torch.nn.ReLU(),
            torch.nn.Upsample(scale_factor=2, mode='bicubic'),
            torch.nn.ConvTranspose2d(32, 16, 5, 1, 2),
            torch.nn.ReLU(),
            torch.nn.Upsample(scale_factor=2, mode='bicubic'),
            torch.nn.ConvTranspose2d(16, 1, 3, 1, 1)
        )
    def forward(self, x):
        x = self.fc1(x)
        x = x.resize(-1, 50, 3, 3)
        x = self.conv1(x)
        return x

In [ ]:

lossFunction = torch.nn.MSELoss()